In [3]:
import sys
import spikeinterface as si
import matplotlib.pyplot as plt
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.widgets as sw
import spikeinterface.qualitymetrics as sqm
import json
import probeinterface

from probeinterface import Probe, ProbeGroup

import os
import numpy as np
from spikeinterface.core import concatenate_recordings

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.stats import pearsonr
import pandas as pd
import numpy as np
from matplotlib.collections import LineCollection


In [7]:
file

'0912_250912_151403气麻撤后后10分钟（能够走动）.rhs'

In [8]:
recording_raw_list

[IntanRecordingExtractor: 16 channels - 30.0kHz - 1 segments - 18,000,000 samples 
                          600.00s (10.00 minutes) - uint16 dtype - 549.32 MiB
   file_path: /media/ubuntu/sda/cen/0912-麻醉觉醒PF记录/0912_250912_145403-气麻撤后30分钟记录/0912_250912_145403-气麻撤后前10分钟.rhs,
 IntanRecordingExtractor: 16 channels - 30.0kHz - 1 segments - 18,000,000 samples 
                          600.00s (10.00 minutes) - uint16 dtype - 549.32 MiB
   file_path: /media/ubuntu/sda/cen/0912-麻醉觉醒PF记录/0912_250912_145403-气麻撤后30分钟记录/0912_250912_150403气麻撤后中间10分钟.rhs]

In [13]:

file_list = os.listdir(f"/media/ubuntu/sda/cen/0912-麻醉觉醒PF记录/0912_250912_145403-气麻撤后30分钟记录")
file_list.remove("settings.xml")
recording_raw_list = []
for file in file_list:
    recording_raw_list.append(se.read_intan(f"/media/ubuntu/sda/cen/0912-麻醉觉醒PF记录/0912_250912_145403-气麻撤后30分钟记录/{file}", stream_id= '0'))
recording_raw = concatenate_recordings(recording_list=recording_raw_list)
recording_raw = spre.unsigned_to_signed(recording_raw)
recording_recorded = recording_raw.remove_channels(['A-001', 'A-003', 'A-005', 'A-007', 'A-009', 'A-011', 'A-013', 'A-015'])
recording_recorded = spre.bandpass_filter(recording_recorded, freq_min=300, freq_max=3000)
recording_recorded = spre.notch_filter(recording_recorded, freq=50)
recording_f = spre.common_reference(recording_recorded, reference="global", operator="median")

In [14]:
from probeinterface import write_probeinterface, read_probeinterface

probe_30channel = read_probeinterface('./probe.json')
probe_30channel.set_global_device_channel_indices([i for i in range(8)])
recording_recorded = recording_f.set_probegroup(probe_30channel)

In [15]:
output_folder = '/media/ubuntu/sda/cen/sorted_results/0912_250912_145403-气麻撤后30分钟记录'
recording_preprocessed = recording_recorded.save(format="binary")
print(recording_preprocessed)
sorting_kilosort4 = ss.run_sorter(sorter_name="kilosort4", recording=recording_preprocessed, folder=output_folder + "/kilosort4")
analyzer_kilosort4 = si.create_sorting_analyzer(sorting=sorting_kilosort4, recording=recording_preprocessed, format='binary_folder', folder=output_folder + '/analyzer_kilosort4_binary')

extensions_to_compute = [
            "random_spikes",
            "waveforms",
            "noise_levels",
            "templates",
            "spike_amplitudes",
            "unit_locations",
            "spike_locations",
            "correlograms",
            "template_similarity"
        ]

extension_params = {
    "unit_locations": {"method": "center_of_mass"},
    "spike_locations": {"ms_before": 0.1},
    "correlograms": {"bin_ms": 0.1},
    "template_similarity": {"method": "cosine_similarity"}
}

analyzer_kilosort4.compute(extensions_to_compute, extension_params=extension_params)

qm_params = sqm.get_default_qm_params()
analyzer_kilosort4.compute("quality_metrics", qm_params)

import spikeinterface.exporters as sexp
sexp.export_to_phy(analyzer_kilosort4, output_folder + "/phy_folder_for_kilosort", verbose=True)

Use cache_folder=/tmp/spikeinterface_cache/tmpdvte61la/G9H16WH8
write_binary_recording 
engine=process - n_jobs=1 - samples_per_chunk=30,000 - chunk_memory=468.75 KiB - total_memory=468.75 KiB - chunk_duration=1.00s


write_binary_recording (no parallelization): 100%|██████████| 1801/1801 [00:13<00:00, 134.22it/s]


BinaryFolderRecording: 8 channels - 30.0kHz - 1 segments - 54,017,792 samples 
                       1,800.59s (30.01 minutes) - int16 dtype - 824.25 MiB


noise_level (no parallelization): 100%|██████████| 20/20 [00:00<00:00, 439.73it/s]
Compute : spike_amplitudes + spike_locations (no parallelization): 100%|██████████| 1801/1801 [00:00<00:00, 1900.85it/s]
extract PCs (no parallelization): 100%|██████████| 1801/1801 [00:11<00:00, 158.11it/s]

Run:
phy template-gui  /media/ubuntu/sda/cen/sorted_results/0912_250912_145403-气麻撤后30分钟记录/phy_folder_for_kilosort/params.py
